In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance

In [ ]:
tf.__version__

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [ ]:
sample_path = "/kaggle/input/state-farm-distracted-driver-detection/sample_submission.csv"
imgs_list_path = "/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv"
train_path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train"

## 1.Check data distribution

In [ ]:
driver_imgs_list = pd.read_csv(imgs_list_path)
driver_imgs_list.head()

In [ ]:
os.listdir(train_path)

In [ ]:
def pair_sort(className,values):
    for j in range(0,len(className)-1):
        for i in range(0,len(className)-1):
            if values[i] > values[i+1]:
                temp =  values[i+1]
                values[i+1] = values[i]
                values[i] = temp

                N_temp =  className[i+1]
                className[i+1] = className[i]
                className[i] = N_temp
    
    return className,values

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(15, 5), dpi=80, facecolor='w', edgecolor='k')

class_names = np.unique(driver_imgs_list['classname'])
class_image_list = [len(driver_imgs_list[driver_imgs_list['classname'] == current_class]) for current_class in class_names]

class_names,class_image_list=  pair_sort(class_names,class_image_list)

#plt.figure()
plt.suptitle('Number of images per Class')
plt.bar(class_names,class_image_list,color=(0.2, 0.3, 0.6, 0.6))
plt.show()

In [ ]:
from matplotlib.pyplot import figure
sub_names = np.unique(driver_imgs_list['subject'])
sub_image_list = [len(driver_imgs_list[driver_imgs_list['subject'] == current_sub]) for current_sub in sub_names]
sub_names,sub_image_list=  pair_sort(sub_names,sub_image_list)

figure(num=None, figsize=(15, 10), dpi=80, facecolor='w', edgecolor='k')

y_pos = np.arange(len(sub_names))
# Create horizontal bars
plt.barh(y_pos, sub_image_list,color=(0.2, 0.4, 0.6, 0.6))
 
# Create names on the y-axis
plt.yticks(y_pos,sub_names )
plt.suptitle('Number of images per subject')

# Show graphic
plt.show()

### Changing load function to augment images with movenet

In [ ]:
def make_square_test(im):
    """
    Adds black pixel padding to an image to make it a square.
    
    Args:
        im: A PIL Image
    """
    x, y = im.size
    size = max(x, y)
    new_im = Image.new('RGB', (size, size))
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

In [ ]:
module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/3")
input_size = 256
    
def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoint_with_scores = outputs['output_0'].numpy()
    return keypoint_with_scores

In [ ]:
# The movenet model can be used to take input images of 256x256 pixels
img_width,img_height = (256,256)
model_input_shape = (img_width,img_height,3)
batch_size = 16
input_image = (img_width, img_height)


def load_image(path):
    read_path = train_path+"/"+path
    
    # getting movenet coordinates
    movenet_image = tf.io.read_file(read_path)
    movenet_image = tf.image.decode_jpeg(movenet_image)
    movenet_image = tf.expand_dims(movenet_image, axis=0)
    movenet_image = tf.image.resize_with_pad(movenet_image, 256, 256)
    
    movenet_coordinates = movenet(movenet_image)
    movenet_coordinates = tf.reshape(movenet_coordinates, [17, 3]).numpy()
    
    # Finding the least and most x, y coordinates in which a limb is detected
    min_x = 1
    min_y = 1
    max_x = 0
    max_y = 0
    
    for coord in movenet_coordinates[:11]:
        if coord[1] < min_x:
            min_x = coord[1]
        if coord[0] < min_y:
            min_y = coord[0]
        if coord[1] > max_x: 
            max_x = coord[1]
        if coord[0] > max_y: 
            max_y = coord[0]
    
    # loading the image again
    image = Image.open(read_path)
    image = make_square_test(image)  
    
    width, height = image.size
    
    # Uses the extreme end coordinates found earlier to crop the images
    min_x = min_x - (min_x)*0.2
    max_x = 1
    min_y = min_y - (min_y)*0.5
    max_y = max_y + (1-max_y)*0.1
    
    # cropping dimensions
    left = math.floor(min_x*width)
    right = math.ceil(max_x*width)
    top = math.floor(min_y*height)
    bottom = math.ceil(max_y*height)
    
    image = image.crop((left, top, right, bottom))
    
    image = make_square_test(image)
    image = image.resize(input_image)
    
    return image

In [ ]:
def show_images(image_ids,class_names):
    pixels = [load_image(path) for path in image_ids]
    
    num_of_images = len(image_ids)
    
    fig, axes = plt.subplots(
        1, 
        num_of_images, 
        figsize=(5 * num_of_images, 5 * num_of_images),
        
    )
   
    
    for i, image_pixels in enumerate(pixels):
        axes[i].imshow(image_pixels)
        axes[i].axis("off")
        axes[i].set_title(class_names[i])

## 2.Plot class (images after augmentation)

In [ ]:
sub_names_imgs = [ current_class+"/"+driver_imgs_list[driver_imgs_list['classname'] == current_class]['img'].values[0] for current_class in class_names]

show_images(sub_names_imgs[:5],class_names[:5])
show_images(sub_names_imgs[5:],class_names[5:])

 ## 3. Loads the labelled images, crops, and saves them as outputs

In [ ]:
train_path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train"
test_path = "/kaggle/input/state-farm-distracted-driver-detection/imgs/test"
output_path = "/kaggle/working/imgs/train"

In [ ]:
for current_class in class_names:
    select_df = driver_imgs_list[driver_imgs_list['classname'] == current_class ]
    image_list = select_df['img'].values
    if not os.path.exists(output_path+"/"+current_class):
        os.makedirs(output_path+"/"+current_class)
    for filename in image_list:
        # load_image(current_class+"/"+filename)
        im = load_image(current_class+"/"+filename)
        im.save(output_path+"/"+current_class+"/"+filename)



In [ ]:
!zip -r /kaggle/working/output.zip /kaggle/working/imgs

In [ ]:
!rm -rf /kaggle/working/imgs/*
!rm -r /kaggle/working/imgs